In [1]:
"""

Author: Sang Yuchen
Date:2020/05/31
Desc: mlp for text classification adversarial examples discriminating

"""

import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#超参数设置
vocab_size=10000
seq_length=80
buffer_length=1000
embedding_dim=100
node_num=(embedding_dim * seq_length + 1) *2/3     #MLP隐藏层节点数

batch_size=32
epochs=10
learning_rate=0.001
n_samples = 4000
total_samples=12500

#数据处理
imdb=keras.datasets.imdb
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=vocab_size)

word_index=imdb.get_word_index()
word_index={k:(v+3) for k,v in word_index.items()}

word_index['<PAD>']=0
word_index['<START>']=1
word_index['<UNK>']=2
word_index['<EOS>']=3

index_word=dict([(value,key)for(key,value)in word_index.items()])

def decode_review(review):
    """

    :param review: 一条评论，是一个整数序列
    :return: 一条评论，是一个英文单词序列
    """
    return ' '.join([index_word.get(word,'<UNK>')for word in review])

#data_with_START_token :
# shape: sample_num,seq_length+1
# Desc: decoder input data
train_data_with_START_token = keras.preprocessing.sequence.pad_sequences(train_data,
           value=word_index['<PAD>'], padding='post', truncating='post', maxlen=seq_length+1)
test_data_with_START_token = keras.preprocessing.sequence.pad_sequences(test_data,
           value=word_index['<PAD>'], padding='post', truncating='post', maxlen=seq_length + 1)


db_train=tf.data.Dataset.from_tensor_slices((train_data_with_START_token,train_labels))

ModuleNotFoundError: No module named 'tensorflow.python.tools'; 'tensorflow.python' is not a package

In [ ]:
ds_label_0 = tf.data.Dataset.from_tensor_slices([data[1:] for data, label in db_train.as_numpy_iterator()
                                           if label == 0])
ds_label_1 = tf.data.Dataset.from_tensor_slices([data[1:]  for data, label in db_train.as_numpy_iterator()
                                           if label == 1])

In [ ]:
ds_label_0 = ds_label_0.shuffle(buffer_size=buffer_length).batch(batch_size=batch_size,drop_remainder=False)
ds_label_1 = ds_label_1.shuffle(buffer_size=buffer_length).batch(batch_size=batch_size,drop_remainder=False)
next(iter(ds_label_0))
